In [1]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Iterable, Any

import numpy as np

from qqe.circuit.spec import CircuitSpec, GateSpec
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

from qqe.backend import QuimbBackend
from qqe.circuit.families import (
    CliffordBrickwork,
    HaarBrickwork,
    QuansistorBrickwork,
    RandomCircuit,
)
from qqe.circuit.gates import clifford_recipe_unitary
from qqe.experiments.plotting import plot_pennylane_circuit
from qqe.GNN import circuit_spec_to_nx_dag


c:\Users\Victor\Desktop\Université\Research\qml-quansistor-entropy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def to_qasm(spec: CircuitSpec, gates: tuple[GateSpec]) -> str:
    lines = []
    lines.append('OPENQASM 2.0;')
    lines.append('include "qelib1.inc";')

    lines.append('opaque qx(a,b,g) q0,q1;')
    lines.append('opaque qy(a,b,g) q0,q1;')
    lines.append('opaque haar(s) q0,q1;')
    lines.append(f"qreg q[{spec.n_qubits}];")

    for gate in gates:
        k = gate.kind
        if k == "H":
            lines.append(f"h q[{gate.wires[0]}];")
        elif k == "S":
            lines.append(f"s q[{gate.wires[0]}];")
        elif k == "T":
            lines.append(f"t q[{gate.wires[0]}];")
        elif k == "I":
            lines.append(f"id q[{gate.wires[0]}];")
        elif k == "CNOT":
            lines.append(f"cx q[{gate.wires[0]}],q[{gate.wires[1]}];")
        elif k == "RX":
            lines.append(f"rx({float(gate.params[0])}) q[{gate.wires[0]}];")
        elif k == "RY":
            lines.append(f"ry({float(gate.params[0])}) q[{gate.wires[0]}];")
        elif k == "RZ":
            lines.append(f"rz({float(gate.params[0])}) q[{gate.wires[0]}];")

        elif k == "quansistor":
            a, b, g, axis = gate.params
            fn = "qx" if str(axis) == "X" else "qy"
            lines.append(f"{fn}({float(a)},{float(b)},{float(g)}) q[{gate.wires[0]}],q[{gate.wires[1]}];")

        elif k == "haar":
            lines.append(f"haar({gate.params[0]}) q[{gate.wires[0]}], q[{gate.wires[1]}];")
        else:
            raise ValueError(f"Unsupported gate: {gate.kind}")
    return "\n".join(lines) + "\n"

In [3]:
family_registry = {
    "haar": HaarBrickwork,
    "clifford": CliffordBrickwork,
    "quansistor": QuansistorBrickwork,
    "random": RandomCircuit,
}

In [4]:
n_qubits = 6
n_layers = 20
seed = 42
family = "quansistor"

In [5]:
circuit = family_registry[family]()
circuit_spec = circuit.make_spec(
    n_qubits=n_qubits,
    n_layers=n_layers,
    d=2,
    seed=seed,
)

In [6]:
qasm = to_qasm(circuit_spec, circuit_spec.gates)

In [7]:
print(qasm)

OPENQASM 2.0;
include "qelib1.inc";
opaque qx(a,b,g) q0,q1;
opaque qy(a,b,g) q0,q1;
opaque haar(s) q0,q1;
qreg q[6];
qx(-0.5832940131378769,0.4888514512501293,0.6839752539010373) q[0],q[1];
qx(0.429431752482278,0.25865959394725,0.4967740224221763) q[2],q[3];
qx(-0.997455994691253,-0.25553839948107626,1.113363475469236) q[4],q[5];
qx(0.15584734044542534,-0.40676936482796916,0.02653009865420029) q[1],q[2];
qx(-0.26220706324493787,-0.47139571119594936,1.4887916085503585) q[3],q[4];
qy(0.7871494650832584,-0.6772803198491009,-0.17167380891536096) q[0],q[1];
qy(-1.199869354120293,2.2357944700819363,0.012607563119570295) q[2],q[3];
qy(0.41022072754393446,-0.649274019109208,0.024457708050798437) q[4],q[5];
qx(-1.2666803487377614,0.9460030594318729,-1.266296585618318) q[1],q[2];
qx(-0.5514815247680959,1.1200109973724706,0.30512850274242426) q[3],q[4];
qy(0.471911420211037,1.0850378767411208,-0.5710088793656896) q[0],q[1];
qx(-0.4753893371779868,0.9209067768841356,1.748636685394089) q[2],q[3];
q

In [8]:
for family in [
    "haar",
    "clifford",
    "quansistor",
    "random",
]:
    circuit = family_registry[family]()
    circuit_spec = circuit.make_spec(
        n_qubits=n_qubits,
        n_layers=n_layers,
        d=2,
        seed=seed,
    )
    qasm = to_qasm(circuit_spec, circuit_spec.gates)
    print(f"--- {family} ---")
    print(qasm)

--- haar ---
OPENQASM 2.0;
include "qelib1.inc";
opaque qx(a,b,g) q0,q1;
opaque qy(a,b,g) q0,q1;
opaque haar(s) q0,q1;
qreg q[6];
haar(5359974671552959166) q[0], q[1];
haar(11674310902546941586) q[2], q[3];
haar(5717523544511513278) q[4], q[5];
haar(14540638928500854118) q[1], q[2];
haar(2638574402333243807) q[3], q[4];
haar(8643831700009403671) q[0], q[1];
haar(5064884065058016338) q[2], q[3];
haar(14941407150017642958) q[4], q[5];
haar(707983443941673481) q[1], q[2];
haar(16622554523054934403) q[3], q[4];
haar(14071833539441583991) q[0], q[1];
haar(11852778261972599031) q[2], q[3];
haar(3510508824664847127) q[4], q[5];
haar(5936474695770602454) q[1], q[2];
haar(13238521039589360075) q[3], q[4];
haar(8203273070920193400) q[0], q[1];
haar(12497179642348666848) q[2], q[3];
haar(6994686035885108658) q[4], q[5];
haar(16228701079618935796) q[1], q[2];
haar(7242864466175550027) q[3], q[4];
haar(2562323464460786870) q[0], q[1];
haar(4926595301799012362) q[2], q[3];
haar(465293583270981720) q

In [9]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_dag

In [11]:
n_qubits = 6
n_layers = 10
seed= 42
family = "random"

In [12]:
circuit = family_registry[family]()
circuit_spec = circuit.make_spec(
    n_qubits=n_qubits,
    n_layers=n_layers,
    d=2,
    seed=seed,
)

In [13]:
gates = circuit_spec.gates
qasm = to_qasm(circuit_spec, gates)

In [14]:
qc = QuantumCircuit.from_qasm_str(qasm)
dag = circuit_to_dag(qc)

In [15]:
fig = qc.draw("mpl", vertical_compression="low", fold=-1)
fig.savefig(f"circuit_{family}.png", dpi=200, bbox_inches="tight")

MissingOptionalLibraryError: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."

In [16]:
qubit_index = {q: i for i, q in enumerate(qc.qubits)}

print("\nTopological order:")
for node in dag.topological_op_nodes():
    wires = [qubit_index[q] for q in node.qargs]
    print(node.name, wires)


Topological order:
ry [0]
rx [0]
rx [0]
ry [0]
rz [1]
ry [2]
cx [2, 1]
ry [1]
rz [2]
cx [2, 1]
rx [1]
ry [2]
cx [1, 2]
ry [1]
cx [0, 1]
rz [0]
ry [0]
ry [0]
ry [0]
ry [1]
rx [2]
rz [3]
rz [3]
ry [4]
ry [4]
cx [4, 3]
rz [3]
rz [4]
cx [3, 4]
rz [3]
cx [2, 3]
rz [2]
cx [2, 1]
ry [1]
rz [2]
cx [1, 2]
rz [1]
rx [1]
cx [1, 0]
rx [0]
rx [0]
ry [1]
rx [1]
rz [2]
rx [3]
ry [4]
ry [5]
rx [5]
rx [5]
rx [5]
cx [5, 4]
rz [4]
cx [4, 3]
ry [3]
ry [4]
cx [3, 4]
rz [3]
cx [3, 2]
rz [2]
ry [2]
ry [3]
rx [4]
rz [5]
ry [5]
rz [5]
cx [5, 4]
ry [4]
cx [4, 3]
ry [3]
cx [3, 2]
rx [2]
ry [3]
cx [2, 3]
rz [4]
ry [5]
rx [5]
cx [4, 5]
rx [4]
ry [5]
cx [5, 4]


In [17]:
from qiskit.visualization import dag_drawer

dag_drawer(dag, filename=f"dag_{family}.png", style="color")

MissingOptionalLibraryError: "The 'Graphviz' library is required to use 'dag_drawer'.  To install, follow the instructions at https://graphviz.org/download/. Qiskit needs the Graphviz binaries, which the 'graphviz' package on pip does not install. You must install the actual Graphviz software."

In [18]:
GATE_TYPE = ["IN", "OUT","H", "S", "T", "RX", "RY", "RZ", "CX", "QX", "QY","HAAR"]

In [19]:
d = len(GATE_TYPE) + n_qubits

In [20]:
node = [0] * d

In [21]:
print(d)

18


In [22]:
node

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [23]:
for i in dag.topological_op_nodes():
    print(i)

2050445228432
2050445228288
2050445228576
2050445228720
2050445228864
2050445229008
2050445229152
2050445229296
2050445229440
2050445229584
2050445229728
2050445229872
2050445384208
2050445377584
2050445377728
2050445377872
2050445378016
2050445378160
2050445378304
2050445378448
2050445378592
2050445378736
2050445378880
2050445379024
2050445379168
2050445379312
2050445379456
2050445379600
2050445379744
2050445379888
2050445380032
2050445380176
2050445380320
2050445380464
2050445380608
2050445380752
2050445380896
2050445381040
2050445381184
2050445381328
2050445381472
2050445381616
2050445381760
2050445381904
2050445382048
2050445382192
2050445382336
2050445382480
2050445382624
2050445382768
2050445382912
2050445383056
2050445383200
2050445383344
2050445383488
2050445383632
2050445383776
2050445383920
2050464315792
2050464308880
2050464301680
2050464301536
2050464303408
2050464303264
2050464301104
2050464301392
2050464301248
2050464301824
2050464301968
2050464302256
2050464302112
205046

In [24]:
for row in node_embedding:
    print(row)

NameError: name 'node_embedding' is not defined